# Extração de dados da Mátricula do Imóvel

O seguinte algoritmo tem como obejivo extrair os dados de uma matrícula de imóvel, a fim de trazer informações mais simplicadas do documento.

In [119]:
# !pip install pdfplumber

import pandas as pd
import numpy as np
import pdfplumber 
import json
import re

### Done till here

In [283]:
def extrair_texto(caminho_pdf: str) -> str:
    """
    Extrai o texto do corpo principal de um PDF, ignorando as margens
    através da bounding box.
    """
    texto_completo = ""
    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            # Dimensões da página
            largura = pagina.width
            altura = pagina.height
            
            # x0:  margem esquerda; top: margem superior; x1: margem direita; bottom: margem inferior
            bounding_box = (0.1 * largura, 0.25 * altura, 0.9 * largura, 0.9 * altura)
            
            # recorte da página no espaço definido
            pagina_recortada = pagina.crop(bounding_box)
            
            texto_pagina = pagina_recortada.extract_text()
            
            if texto_pagina:
                texto_completo += texto_pagina + "\n"
            
    # normalização de espaços
    texto = re.sub(r'\s+', ' ', texto_completo)
    blocos = re.sub(r'[Dd]ou f[eé].', '\n', texto).split('\n')

    return [b.strip() for b in blocos if b.strip()]

In [282]:
def extrair_composicao(descricao: str) -> str:
    """
    Extrai a descrição da composição do imóvel, lidando com a ordem
    variável das informações de área e comodidades.
    """
    composicao_texto = ""
    
    # "contendo" é o que indica o que vem primeiro
    match_contendo = re.search(r'contendo\s+(.*?)(?:\s*PROPRIETÁRIO|\s*TÍTULO ANTERIOR:)', descricao, re.IGNORECASE | re.DOTALL)
    
    # Padrao A:  áreas depois composição 
    if match_contendo:
        # Se encontrou "contendo", a composição é o que vem depois
        composicao_texto = match_contendo.group(1)
    else:
        # Padrão B: composição depois áreas
        # Captura tudo desde até encontrar "área" | fim do bloco
        match_padrao_a = re.search(r'^(.*?)(?:\s+com\s+área|\s*PROPRIETÁRIO)', descricao, re.IGNORECASE | re.DOTALL)
        if match_padrao_a:
            composicao_texto = match_padrao_a.group(1)
            
    return " ".join(composicao_texto.strip().split())

In [ ]:
def extrair_proprietario_inicial(bloco_descricao: str) -> dict:
    """
    Extrai de forma estruturada as informações do proprietário inicial
    do imóvel a partir do bloco de descrição.
    """
    proprietario_dados = {}
    
    bloco_prop_match = re.search(r'PROPRIET[ÁA]RIO:\s*(.*?)\s*T[IÍ]TULO ANTERIOR:', bloco_descricao, re.DOTALL | re.IGNORECASE)
    
    if not bloco_prop_match:
        return None 
    
    texto_prop = bloco_prop_match.group(1)
    
    # nome do proprietário (até a primeira ", ")
    nome_match = re.search(r'^(.*?)(?:,\s)', texto_prop)
    if nome_match:
        proprietario_dados["nome"] = nome_match.group(1).strip().rstrip(',')
    
    # tipo e o número do identificador (CNPJ/CGC/CPF)
    id_match = re.search(r'inscrita no\s+([A-Z\/]+)[\s\S]*?(\d[\d\.\-\/]+)', texto_prop, re.IGNORECASE)
    if id_match:
        proprietario_dados["identificador"] = f"{id_match.group(1).strip()} {id_match.group(2).strip()}" 
        
    return proprietario_dados if proprietario_dados else None

### Continue

In [ ]:

def parse_descricao(bloco: str) -> dict:
    """Extrai dados principais do imóvel e proprietário inicial."""

    dados = {}

# ======================================================================

    # matrícula
    matricula = re.search(r'[Mm]atr[ií]cula\s*(\d{1,3}\.?\d{3})', bloco, re.IGNORECASE)
    if matricula: 
        dados["matricula"] = matricula.group(1)


    # cartório
    cartorio = re.search(r'Registro Geral do (.*?)\)', bloco, re.IGNORECASE)
    if cartorio: dados["cartorio"] = cartorio.group(1)


    # número de Ordem e data de abertura
    ordem = re.search(r'ordem\s*(\d{1,3}\.?\d{3}).*?data de\s*(\d{1,2}.*?\d{4})', bloco, re.IGNORECASE)
    if ordem:
        dados["ordem"] = ordem.group(1)
        dados["data_abertura"] = ordem.group(2)
        

# ======================================================================
    # Transcrição
    padrao_geral = r'Transcri[cç][aã]o:\s*(.*?),\s*situado\s*[aàá]?\s*(.*?),\s*nesta cidade'
    match_geral = re.search(padrao_geral, bloco, re.IGNORECASE | re.DOTALL)

    if match_geral:
        dados.setdefault("imovel", {})
        tipo_descricao = match_geral.group(1).strip()
        dados["imovel"]["tipo"] = " ".join(tipo_descricao.split())
        
        endereco = match_geral.group(2).strip()
        dados["imovel"]["endereco"] = " ".join(endereco.split())

# ======================================================================
    # Áreas
    padroes_area = [
        ("privativa", r'(?:privativa real|real privativa).*?([\d\.,]+)'),
        ("uso_comum", r'(?:uso comum real|real de uso comum).*?([\d\.,]+)'),
        ("total", r'real total (?:de)?\s*([\d\.,]+)'),
        ("privativa_acessoria", r'privativa acess[oó]ria.*?([\d\.,]+)'),
        ("privativa_total", r'privativa total\s*([\d\.,]+)'),
        ("equiv_constr_tt", r'constru[cç][aã]o total (?:de)?\s*([\d\.,]+)'),
        ("fração_ideal", r'fra[cç][aã]o ideal (?:de)?\s*([\d\.,]+)'),
        ("coef_proporcionalidade", r'coeficiente de proporcionalidade\s*([\d\.,]+)'),
        ("cota_terreno", r'cota ideal do terreno (?:de)?\s*([\d\.,]+)')
    ]

    # composição do imóvel
    desc_geral_match = re.search(r'compost[ao]\s+de:(.*?)(?:PROPRIETÁRIO:)', bloco, re.IGNORECASE | re.DOTALL)
    
    if desc_geral_match:
        bloco_descricao_geral = desc_geral_match.group(1)
        areas = {}
        # Busca de todas as áreas
        for chave, area in padroes_area:
            match = re.search(area, bloco_descricao_geral, re.IGNORECASE)
            if match:
                valor_numerico = float(match.group(1).replace('.', '').replace(',', '.'))
                areas[chave] = valor_numerico

        if areas: dados["imovel"]["areas"] = areas

        # Busca da composição do imóvel
        composicao = extrair_composicao(bloco_descricao_geral)

        if composicao: dados["imovel"]["composicao"] = composicao
# ======================================================================

    proprietario = extrair_proprietario_inicial(bloco)
    if proprietario:
        dados["proprietario_inicial"] = proprietario
        
    return dados

### Later

In [ ]:
def parse_ato(bloco: str) -> dict:
    """Extrai dados de um ato (R- ou AV-)."""
    ato = {}
    # Identificador (R- ou AV-)
    m = re.match(r'(R|AV)-\d+-\d+', bloco)
    if m: ato["ato"] = m.group(0)
    # Data
    d = re.search(r'em\s*(\d{1,2}.*?\d{4})', bloco)
    if d: ato["data"] = d.group(1)
    # Tipo (heurística)
    if "compra" in bloco.lower() and "venda" in bloco.lower():
        ato["tipo"] = "Compra e venda"
    elif "alien" in bloco.lower():
        ato["tipo"] = "Alienação fiduciária"
    elif "hipotec" in bloco.lower():
        ato["tipo"] = "Hipoteca"
    elif "patrimônio de afetação" in bloco.lower():
        ato["tipo"] = "Patrimônio de afetação"
    elif "consolidação" in bloco.lower():
        ato["tipo"] = "Consolidação da propriedade"
    elif "cancelamento" in bloco.lower():
        ato["tipo"] = "Cancelamento"
    else:
        ato["tipo"] = "Outro"
    # Valores
    valores = re.findall(r'R\$ ?[\d\.\,]+', bloco)
    if valores: ato["valores"] = valores
    return ato


# Exemplo de uso:
# arquivo = "../input/1444404404084.pdf"
# arquivo = "../input/1444411417161.pdf"
arquivo = "../input/1600000131788.pdf"
resultado = processar_matricula(arquivo)
print(json.dumps(resultado, indent=2, ensure_ascii=False))

{
  "imovel": {
    "matricula": "87.950",
    "cartorio": "2º Ofício do Registro de Imóveis Zona Norte",
    "ordem": "87.950",
    "data_abertura": "26 de março de 2010",
    "imovel": {
      "endereco": "situado a Rua Bacharel José de Oliveira Curchatuz, nº 527, esquina com a Rua Vanja Viana Sales, no bairro Aeroclube",
      "tipo": "APARTAMENTO n.º 1802",
      "areas": {
        "privativa": "234,84 m²",
        "uso_comum": "129,11 m²",
        "total": "363,95 m²",
        "construcao_total": "280,20 m²",
        "fração_ideal": "2,18",
        "cota_terreno": "64,61 m²"
      }
    },
    "proprietario_inicial": "Firma CONSTRUTORA MASHIA LTDA., com sede nesta Capital, à Rua João Câncio da Silva, n.° 1.676, sala 203, no bairro de Manaíra, inscrita no CGC n.° 04.534.882/0001-26, neste ato representada pelo sócio Antonio Hugo da Cunha Ximenes, brasileiro, casado, CI n.° 1.227.616 SSP/PB e CPF n.° 601.976.764-72, residente nesta Capital. "
  },
  "historico": [
    {
      "ato":

### HERE

In [295]:
def processar_matricula(caminho_pdf: str) -> dict:
    blocos = extrair_texto(caminho_pdf)
    estrutura = parse_descricao(blocos[0])

    # for bloco in blocos[1:]:
    #     estrutura["historico"].append(parse_ato(bloco))
    
    return estrutura

# Exemplo de uso:
arquivo = "../input/2455251.pdf"
arquivo = "../input/1444408441014.pdf"
arquivo = "../input/1444404404084.pdf"
arquivo = "../input/1444411417161.pdf"
arquivo = "../input/1600000131788.pdf"
resultado = processar_matricula(arquivo)
print(json.dumps(resultado, indent=2, ensure_ascii=False))

{
  "matricula": "87.950",
  "cartorio": "2º Ofício do Registro de Imóveis Zona Norte",
  "ordem": "87.950",
  "data_abertura": "26 de março de 2010",
  "imovel": {
    "tipo": "APARTAMENTO n.º 1802, do EDIFICIO MANOÁ RESIDENCE",
    "endereco": "Rua Bacharel José de Oliveira Curchatuz, nº 527, esquina com a Rua Vanja Viana Sales, no bairro Aeroclube",
    "areas": {
      "privativa": 234.84,
      "uso_comum": 129.11,
      "total": 363.95,
      "equiv_constr_tt": 280.2,
      "fração_ideal": 2.18,
      "cota_terreno": 64.61
    },
    "composicao": "Varanda, sala de estar, sala de jantar, estar intimo, lavabo, quatro suítes sendo uma máster com varanda, cozinha, despensa, área de serviço, dependência de empregada e quatro vagas de garagem, sendo duas cobertas e duas descobertas,"
  },
  "proprietario_inicial": {
    "nome": "Firma CONSTRUTORA MASHIA LTDA.",
    "identificador_tipo": "CGC 04.534.882/0001-26"
  }
}


In [55]:
import pdfplumber
import re
import json
from pathlib import Path



def normalizar_moeda(valor):
    if valor:
        return valor.replace(".", "").replace(",", ".").strip()
    return None

def parse_matricula(texto):
    """Parser simplificado da matrícula de imóvel"""
    dados = {
        "matricula": None,
        "cartorio": {
            "nome": None,
            "oficio": None,
            "comarca": None,
            "cidade": None,
            "estado": None
        },
        "dados_iniciais": {
            "ordem": None,
            "data_abertura": None,
            "descricao_imovel": {},
            "proprietario_inicial": {},
            "titulo_anterior": None
        },
        "historico": []
    }

    # matrícula
    m = re.search(r"Matr[ií]cula\s*(\d+)", texto, re.IGNORECASE)
    if m:
        dados["matricula"] = m.group(1)

    # data de abertura (primeira data formal encontrada)
    d = re.search(r"(\d{1,2} de [A-Za-zçãé]+ de \d{4})", texto)
    if d:
        dados["dados_iniciais"]["data_abertura"] = d.group(1)

    # descrição do imóvel
    desc = re.search(r"Transcrição:(.*?)(?=PROPRIET[ÁA]RIO:)", texto, re.DOTALL)
    if desc:
        dados["dados_iniciais"]["descricao_imovel"]["texto_bruto"] = desc.group(1).strip()

    # proprietário inicial
    prop = re.search(r"PROPRIET[ÁA]RIO:\s*(.*?)(?=T[ÍI]TULO ANTERIOR:)", texto, re.DOTALL)
    if prop:
        dados["dados_iniciais"]["proprietario_inicial"]["texto_bruto"] = prop.group(1).strip()

    # título anterior
    titulo = re.search(r"T[ÍI]TULO ANTERIOR:\s*(.*)", texto)
    if titulo:
        dados["dados_iniciais"]["titulo_anterior"] = titulo.group(1).strip()

    # histórico de atos (R- ou AV-)
    atos = re.findall(r"((?:R|AV)-\d+.*?)(?=(?:R|AV)-\d+|$)", texto, re.DOTALL)
    for ato in atos:
        cab = re.match(r"((?:R|AV)-\d+)", ato.strip())
        dados["historico"].append({
            "ato": cab.group(1) if cab else None,
            "texto_bruto": ato.strip()
        })

    return dados

def processar_pdf(pdf_path, saida_json):
    texto_pdf = extrair_texto(pdf_path)
    dados_json = parse_matricula(texto_pdf)

    with open(saida_json, "w", encoding="utf-8") as f:
        json.dump(dados_json, f, ensure_ascii=False, indent=2)

    return dados_json

# ====== Exemplo de uso ======
pdf1 = Path("../input/1444404404084.pdf")   # matrícula 61.080
# pdf2 = Path("1444411417161.pdf")   # matrícula 115.751

dados1 = processar_pdf(pdf1, "matricula_61080.json")
# dados2 = processar_pdf(pdf2, "matricula_115751.json")

print("Extração concluída. Arquivos JSON gerados.")


Extração concluída. Arquivos JSON gerados.


In [ ]:
dados1